In [5]:
import numpy as np
import matrix_dev
import matplotlib.pyplot as plt
import _pickle
import gensim
import gensim.corpora as corpora
import os
from pprint import pprint
from gensim.models import CoherenceModel
from nltk.corpus import stopwords
import spacy
import pandas as pd
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt 
%matplotlib inline
from lda_implementation import lda_model
import pandas as pd
from scipy.sparse import csr_matrix
import warnings
warnings.filterwarnings('ignore')

In [2]:
dominant_topic = pd.read_csv('../../feature_groups/lda_distributions/dominant_topic.csv')
representative_doc = pd.read_csv('../../feature_groups/lda_distributions/representative_doc.csv')
topic_dist_doc = pd.read_csv('../../feature_groups/lda_distributions/topic_dist_doc.csv')

In [6]:
lda = lda_model()
lda.load_data('raw_docs', 'dev')

loading pickles


In [4]:
name = 'word_topics'
path = os.path.join('../../feature_groups/lda_pickles', name)
with open(path, "rb") as f:
	word_topics = _pickle.load(f)

In [7]:
docs = lda.raw_docs
vocab = lda.id2word.values()

In [8]:
vocab = set(vocab)

In [47]:
corp =  lda.corpus
matrix = np.zeros((10000, len(vocab)))
for r, row in enumerate(corp):
    for tup in row:
        matrix[r][tup[0]] = 1
matrix

array([[1., 1., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 1., 1.]])

array([1., 1., 1., ..., 0., 0., 0.])

In [48]:
def word_topic_assign(word_topics, word_count_doc):
    """
    word_topic -- k times |V|
    word_count_doc -- d times |V|
    """
    all_matrices = []
    for doc_indx, doc in enumerate(word_count_doc):
        matrix = csr_matrix(((np.array(word_topics)*doc) > 0).astype(int))
        all_matrices.append(matrix)
    return matrix

In [49]:
word_topic_assign =  word_topic_assign(word_topics, matrix)

In [50]:
def format_word_topic_assign(word_topic_assign):
    ret_list = []
    for doc_num, doc in enumerate(word_topic_assign):
        for row, col in zip(*doc.nonzero()):
            ret_list.append([doc_num, row, col])
    return pd.DataFrame(ret_list)

In [51]:
format_word_topic = format_word_topic_assign(word_topic_assign)

In [52]:
word_topic_assign


<85x19326 sparse matrix of type '<class 'numpy.int64'>'
	with 3 stored elements in Compressed Sparse Row format>

In [41]:
for doc_indx in np.array(word_count_doc):
    print(doc_indx)

TypeError: iteration over a 0-d array

In [44]:
word_count_doc.todense()

matrix([[1., 1., 1., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 1., 1., 1.]])

In [13]:
word_topic_assign

<85x19326 sparse matrix of type '<class 'numpy.int64'>'
	with 3 stored elements in Compressed Sparse Row format>

In [ ]:
def word_MRF(matrix):
    (M, N) = matrix.shape
    old = matrix
    new_matrix = np.zeros((M,N))
    while (SNR(old, new_matrix)>0.01):
        print(SNR(old, new_matrix))
        for i in range(M):
            for j in range(N):
                index = neighbour(i,j,M,N)
                a = cost(1,matrix[i,j], old,index)
                b = cost(0,matrix[i,j], old,index)
                if a>b:
                    new_matrix[i][j]=1
                else:
                    new_matrix[i][j]=0
        old = new_matrix
    print(SNR(old, new_matrix))
    return new_matrix

In [ ]:
def SNR(A,B):
    if A.shape == B.shape:
        return np.sum(np.abs(A-B))/A.size
    else:
        raise Exception("not equal size")

In [ ]:
def neighbour(i,j,M,N):
    